In [22]:
import pandas as pd
import sqlite3

In [30]:
from sqlalchemy.sql import expression, functions
from sqlalchemy import types

In [3]:
%run 'LocalSQLConnection.py'

2.6.0
2. Query all tasks
(0, 'Tennis Court 1', 5, 25, 10000, 200)
(1, 'Tennis Court 2', 5, 25, 8000, 200)
(2, 'Badminton Court', 0, 15.5, 4000, 50)
(3, 'Table Tennis', 0, 5, 320, 10)
(4, 'Massage Room 1', 9.9, 80, 4000, 3000)
(5, 'Massage Room 2', 9.9, 80, 4000, 3000)
(6, 'Squash Court', 3.5, 17.5, 5000, 80)
(7, 'Snooker Table', 0, 5, 450, 15)
(8, 'Pool Table', 0, 5, 400, 15)


In [23]:
conn = sqlite3.connect("sqlite_db_pythonsqlite.db")

In [24]:
cur = conn.cursor()

In [6]:
cur.execute("SELECT * FROM (SELECT sub.facility, SUM( sub.cost ) AS total_revenue FROM (SELECT Facilities.name AS facility, CASE WHEN Bookings.memid =0 THEN Facilities.guestcost * Bookings.slots ELSE Facilities.membercost * Bookings.slots END AS cost FROM Bookings INNER JOIN Facilities ON Bookings.facid = Facilities.facid INNER JOIN Members ON Bookings.memid = Members.memid)sub GROUP BY sub.facility)sub2 WHERE sub2.total_revenue <1000;")

In [7]:
results = cur.fetchall()
print(results)

[('Pool Table', 270), ('Snooker Table', 240), ('Table Tennis', 180)]


Question:10 /* Q11: Produce a report of members and who recommended them in alphabetic surname,firstname order */

In [58]:
mem_reco = "SELECT (firstname ||' '|| surname) as Mem_name, (SELECT firstname || ' ' || surname from Members as m2 where m2.memid = m1.recommendedby and m1.recommendedby != 0) as Reco_name FROM Members as m1 WHERE memid != 0 ORDER BY Reco_name"

In [60]:
#list of the Member with their recommenders
pd.read_sql_query(mem_reco, conn).head()

,Mem_name,Reco_name
0,Darren Smith,None
1,Tracy Smith,None
2,Tim Rownam,None
3,Burton Tracy,None
4,Jemima Farrell,None


 Q12: Find the facilities with their usage by member, but not guests 

In [63]:
facili_usage = "SELECT f.name, count(*) as Usage FROM Facilities as f INNER JOIN Bookings as b on b.facid = f.facid where b.memid != 0 group by f.name"

In [64]:
pd.read_sql_query(facili_usage, conn).head()

,name,Usage
0,Badminton Court,344
1,Massage Room 1,421
2,Massage Room 2,27
3,Pool Table,783
4,Snooker Table,421


 Q13: Find the facilities usage by month, but not guests */

In [69]:
fac_month ='''SELECT f.name, (strftime('%Y-%m', starttime)) as month_count, count(*) as usage_count 
        FROM Facilities as f INNER JOIN Bookings as b on b.facid = f.facid 
        where b.memid != 0 group by f.name, month_count'''

In [70]:
pd.read_sql_query(fac_month, conn).head()

,name,month_count,usage_count
0,Badminton Court,2012-07,51
1,Badminton Court,2012-08,132
2,Badminton Court,2012-09,161
3,Massage Room 1,2012-07,77
4,Massage Room 1,2012-08,153
